In [1]:
import os
import matplotlib
import numpy as np
import pandas as pd
import csv
import datetime
from collections import Counter

pd.set_option('display.max_columns', 1000)  # or 1000
pd.set_option('display.max_rows', 1000)  # or 1000
pd.set_option('display.max_colwidth', None) 
pd.options.plotting.backend = "plotly"

In [2]:
directory = "API-logs/"

### Server logs

In [3]:
with open(directory+'server-19days.log','r' , errors='replace')  as f:
    lines = f.readlines() # readlines creates a list of the lines
len(lines)

2722543

In [7]:
requestdf = pd.DataFrame(columns = ['date', 'requests'])

In [8]:
import re
count=0
for line in lines:
    if "POST /generate HTTP/1.1" in line:
        time = line.split('[')[1].split(']')[0]
        code = line.split("POST /generate HTTP/1.1")[1]
        code= re.sub("[^0-9]", "", code)
        code= code.replace('0400','400')
        requestdf.loc[count] = [time, code]
        count+=1

In [9]:
requestdf['date'] = pd.to_datetime(requestdf['date'] , format= "%d/%b/%Y %H:%M:%S")

In [10]:
requestdf.head()

,date,requests
0,2022-09-22 15:15:50,200
1,2022-09-22 15:15:51,200
2,2022-09-22 15:15:55,200
3,2022-09-22 15:16:03,200
4,2022-09-22 15:16:08,200


In [11]:
requestdf['requests'] = pd.to_numeric(requestdf['requests'])

### Total number of requests

In [13]:
requestdf.requests.count()

230768

### Getting total number of days (+dates)

In [14]:
requestdaydf = requestdf.groupby(pd.Grouper(key = 'date', freq='1440min')).count()
print('there are '+str(requestdaydf.shape[0]) + ' days of logs')

there are 18 days of logs


In [15]:
requestdaydf.head(18)

,requests
date,
2022-09-22,9979
2022-09-23,13908
2022-09-24,9491
2022-09-25,12997
2022-09-26,16420
2022-09-27,17521
2022-09-28,14446
2022-09-29,12187
2022-09-30,8524


In [16]:
requestdf.requests.value_counts()

200    224889
400      5523
503       343
500        13
Name: requests, dtype: int64

200 = OK we answered

400 = used bad request

503 = refused because no more room in the queue

### Average number of requests per hour

In [20]:
requesthourdf = requestdf.groupby(pd.Grouper(key = 'date', freq='60min')).count()
requesthourdf.shape

(413, 1)

In [21]:
requesthourdf.head()

,requests
date,
2022-09-22 15:00:00,1338
2022-09-22 16:00:00,1609
2022-09-22 17:00:00,1426
2022-09-22 18:00:00,1544
2022-09-22 19:00:00,1580


In [22]:
requesthourdf.requests.plot()

In [23]:
requesthourdf.tail()

,requests
date,
2022-10-09 15:00:00,93
2022-10-09 16:00:00,322
2022-10-09 17:00:00,884
2022-10-09 18:00:00,889
2022-10-09 19:00:00,125


### Number of requests per 10 minutes

In [65]:
requesthourdf = requestdf.groupby(pd.Grouper(key = 'date', freq='10min')).count()
requesthourdf['time'] = requesthourdf.index
requesthourdf.head()

,requests,time
date,,
2022-09-22 15:10:00,85,2022-09-22 15:10:00
2022-09-22 15:20:00,237,2022-09-22 15:20:00
2022-09-22 15:30:00,237,2022-09-22 15:30:00
2022-09-22 15:40:00,242,2022-09-22 15:40:00
2022-09-22 15:50:00,537,2022-09-22 15:50:00


In [66]:
requesthourdf.shape

(2472, 2)

In [67]:
requesthourdf.tail()

,requests,time
date,,
2022-10-09 18:20:00,133,2022-10-09 18:20:00
2022-10-09 18:30:00,174,2022-10-09 18:30:00
2022-10-09 18:40:00,160,2022-10-09 18:40:00
2022-10-09 18:50:00,213,2022-10-09 18:50:00
2022-10-09 19:00:00,125,2022-10-09 19:00:00


In [68]:
requesthourdf['requests'] = pd.to_numeric(requesthourdf['requests'])

### Code Carbon logs

In [28]:
carbondf = pd.DataFrame(columns = ['date', 'timestamp', 'RAM energy', 'GPU energy', 'GPU power', 'CPU energy', 'Energy consumed'])

In [29]:
## 72 hours

In [30]:
ins=[]
with open(directory+'carbon.log' , 'r') as f:
    lines = f.readlines() # readlines creates a list of the lines
    for line in lines:
        if 'Sep' in line:
            ins.append(lines.index(line))
len(lines)

143138

In [31]:
alldates, sep22, sep23, sep24, sep25, sep26= [], [], [], [], [], []
sep22 = lines[ins[0]:ins[1]]
print(len(sep22))
alldates.append(sep22)
sep23 = lines[ins[1]:ins[2]]
print(len(sep23))
alldates.append(sep23)
sep24 = lines[ins[2]:ins[3]]
alldates.append(sep24)
print(len(sep24))
sep25 = lines[ins[3]:ins[4]]
alldates.append(sep25)
print(len(sep25))
sep26 = lines[ins[4]:len(lines)]
alldates.append(sep26)
print(len(sep26))

13408
36577
36573
36577
20003


In [ ]:
for i in range(1, len(lines), 4):
    for line in lines[i:i + 4]:
        try:
            time = line.split('@')[1].split(']')[0].strip()
        except:
            continue
        if 'RAM :' in line: 
            ram_nrj = line.split('Energy consumed for RAM :')[1].split('RAM')[0].replace('kWh.','')
        elif 'GPUs'in line:
            gpu_nrj = line.split('Energy consumed for all GPUs :')[1].split('All GPUs')[0].replace('kWh.','')
            gpu_power = line.split('All GPUs Power : ')[1].replace('W','').strip()
        elif 'CPUs' in line:
            cpu_nrj= line.split('Energy consumed for all CPUs :')[1].split('All CPUs')[0].replace('kWh.','')
        elif ' electricity used since the begining.' in line:
            total_nrj = line.split(' electricity used since the begining.')[0].split(']')[1].replace(' of','').replace('kWh','')
    carbondf.loc[count] = [time, ram_nrj, gpu_nrj, gpu_power, cpu_nrj, total_nrj]
    count+=1

In [35]:
carbondf = pd.read_csv(directory+'19days_carbon.csv')
#carbondf= carbondf.drop('Unnamed: 0', axis=1)

In [36]:
carbondf.shape

(173636, 7)

In [37]:
carbondf.head()

,date,timestamp,RAM energy,GPU energy,GPU power,CPU energy,Energy consumed
0,2022-09-22,15:12:12,0.002708,0.006703,2412.392,0.000125,0.008222
1,2022-09-22,15:12:22,0.004022,0.013103,2446.520,0.000243,0.016054
2,2022-09-22,15:12:32,0.005333,0.021320,3138.044,0.000361,0.025703
3,2022-09-22,15:12:42,0.006658,0.024947,1389.091,0.000478,0.030758
4,2022-09-22,15:12:52,0.007968,0.028631,1395.522,0.000597,0.035886


In [279]:
## with date : for shorter logs (too much time for 19 days)

In [ ]:
count=0
for day in alldates: 
    date= day[0].replace('\n','')
    print(date)
    print(len(day))
    for i in range(1, len(day), 4):
        for line in day[i:i + 4]:
            try:
                time = line.split('@')[1].split(']')[0].strip()
            except:
                continue
            if 'RAM :' in line: 
                ram_nrj = line.split('Energy consumed for RAM :')[1].split('RAM')[0].replace('kWh.','')
            elif 'GPUs'in line:
                gpu_nrj = line.split('Energy consumed for all GPUs :')[1].split('All GPUs')[0].replace('kWh.','')
                gpu_power = line.split('All GPUs Power : ')[1].replace('W','').strip()
            elif 'CPUs' in line:
                cpu_nrj= line.split('Energy consumed for all CPUs :')[1].split('All CPUs')[0].replace('kWh.','')
            elif ' electricity used since the begining.' in line:
                total_nrj = line.split(' electricity used since the begining.')[0].split(']')[1].replace(' of','').replace('kWh','')
        carbondf.loc[count] = [date, time, ram_nrj, gpu_nrj, gpu_power, cpu_nrj, total_nrj]
        count+=1

In [38]:
carbondf["Energy consumed"] = pd.to_numeric(carbondf["Energy consumed"])
carbondf["GPU power"] = pd.to_numeric(carbondf["GPU power"])

In [39]:
carbondf.shape

(173636, 7)

In [40]:
carbondf[carbondf["GPU power"] > 5000].head()

,date,timestamp,RAM energy,GPU energy,GPU power,CPU energy,Energy consumed
137,2022-09-22,15:33:52,0.174261,0.769212,5066.000,0.015473,0.957637
160,2022-09-22,15:37:22,0.201870,0.927970,5658.650,0.017952,1.131657
177,2022-09-22,15:40:12,0.224245,1.050833,5686.439,0.019958,1.278797
190,2022-09-22,15:42:02,0.238719,1.130143,5014.558,0.021376,1.375667
239,2022-09-22,15:49:52,0.300566,1.453894,5607.181,0.026924,1.781384


In [41]:
#carbondf.date.value_counts()

In [219]:
#carbondf = carbondf.sort_values('Energy consumed')
#carbondf = carbondf.reset_index(drop=True)

In [42]:
carbondf['time'] = carbondf['date'] + ' ' + carbondf['timestamp'].astype(str)

In [43]:
carbondf['time'] = pd.to_datetime(carbondf['time'] , format= "%Y-%m-%d %H:%M:%S")

In [44]:
carbondf.head()

,date,timestamp,RAM energy,GPU energy,GPU power,CPU energy,Energy consumed,time
0,2022-09-22,15:12:12,0.002708,0.006703,2412.392,0.000125,0.008222,2022-09-22 15:12:12
1,2022-09-22,15:12:22,0.004022,0.013103,2446.520,0.000243,0.016054,2022-09-22 15:12:22
2,2022-09-22,15:12:32,0.005333,0.021320,3138.044,0.000361,0.025703,2022-09-22 15:12:32
3,2022-09-22,15:12:42,0.006658,0.024947,1389.091,0.000478,0.030758,2022-09-22 15:12:42
4,2022-09-22,15:12:52,0.007968,0.028631,1395.522,0.000597,0.035886,2022-09-22 15:12:52


In [36]:
#carbondf['date']= carbondf.index

## Energy consumption by 10 min

In [123]:
carbonhourdf = carbondf.groupby(pd.Grouper(key = 'time', freq='10min')).last()
carbonhourdf['time'] = carbonhourdf.index
carbonhourdf.head()

,date,timestamp,RAM energy,GPU energy,GPU power,CPU energy,Energy consumed,time
time,,,,,,,,
2022-09-22 15:10:00,2022-09-22,15:19:52,0.063238,0.271815,2527.826,0.005555,0.332647,2022-09-22 15:10:00
2022-09-22 15:20:00,2022-09-22,15:29:52,0.142635,0.615689,2395.379,0.012757,0.771081,2022-09-22 15:20:00
2022-09-22 15:30:00,2022-09-22,15:39:52,0.221611,1.029444,2541.230,0.019722,1.262780,2022-09-22 15:30:00
2022-09-22 15:40:00,2022-09-22,15:49:52,0.300566,1.453894,5607.181,0.026924,1.781384,2022-09-22 15:40:00
2022-09-22 15:50:00,2022-09-22,15:59:52,0.379544,1.842744,2494.936,0.033890,2.248372,2022-09-22 15:50:00


In [124]:
carbonhourdf['Energy by interval'] = carbonhourdf["Energy consumed"].diff()

In [125]:
#2022-09-22 15:10:00
#2022-10-09 19:00:00 	

In [126]:
carbonhourdf = carbonhourdf.iloc[:-263, :]

In [127]:
carbonhourdf.tail()

,date,timestamp,RAM energy,GPU energy,GPU power,CPU energy,Energy consumed,time,Energy by interval
time,,,,,,,,,
2022-10-09 18:20:00,2022-10-09,18:29:50,195.608696,649.189546,2524.044,17.479743,862.277986,2022-10-09 18:20:00,0.454074
2022-10-09 18:30:00,2022-10-09,18:39:59,195.688778,649.607310,2532.620,17.486826,862.774974,2022-10-09 18:30:00,0.496988
2022-10-09 18:40:00,2022-10-09,18:49:50,195.766173,650.009647,2488.633,17.493910,863.269730,2022-10-09 18:40:00,0.494756
2022-10-09 18:50:00,2022-10-09,18:59:59,195.846260,650.424012,2521.772,17.500994,863.763391,2022-10-09 18:50:00,0.493661
2022-10-09 19:00:00,2022-10-09,19:09:50,195.923680,650.817364,2454.498,17.508077,864.249121,2022-10-09 19:00:00,0.485730


In [128]:
carbonhourdf.shape

(2472, 9)

In [129]:
requesthourdf.shape

(2472, 2)

### Merging the carbon and request DFs

In [130]:
alllogdf = pd.concat([carbonhourdf,requesthourdf], axis=1)
alllogdf['date']= alllogdf.index
alllogdf =alllogdf.dropna()
alllogdf.shape

(2471, 11)

In [132]:
alllogdf_diff.tail()

,date,timestamp,RAM energy,GPU energy,GPU power,CPU energy,Energy consumed,time,Energy by interval,requests,time
2022-09-22 15:20:00,2022-09-22 15:20:00,15:29:52,0.142635,0.615689,2395.379,0.012757,0.771081,2022-09-22 15:20:00,0.438434,237,2022-09-22 15:20:00
2022-10-09 19:00:00,2022-10-09 19:00:00,19:09:50,195.923680,650.817364,2454.498,17.508077,864.249121,2022-10-09 19:00:00,0.485730,125,2022-10-09 19:00:00


In [134]:
alllogdf['requests'] = pd.to_numeric(alllogdf['requests'])

### Tons of carbon

In total

In [135]:
((alllogdf.iloc[-1]['Energy consumed'] * 394.0) /1e6)*1000

340.514153674

Per day

In [136]:
((alllogdf.iloc[-1]['Energy consumed'] * 394.0) /1e6)/18*1000

18.91745298188889

### Plotting raw values

In [137]:
alllogdf["GPU power"].describe()

count    2471.000000
mean     1666.616346
std       549.444911
min      1294.404000
25%      1310.623500
50%      1326.964000
75%      2387.353500
max      5607.181000
Name: GPU power, dtype: float64

In [138]:
fig1= alllogdf.plot(x="date", y=["GPU power"], kind="line")
fig1.update_layout(
    xaxis_title= "Date",
    yaxis_title="GPU Power Used (W)",
)

In [139]:
max_grouping = carbondf.groupby(pd.Grouper(key = 'time', freq='10min')).max()
max_grouping['date']= max_grouping.index
fig2= max_grouping.plot(x="date", y=["GPU power"], kind="line")
fig2.update_layout(
    xaxis_title= "Date",
    yaxis_title="GPU Power Used (W)",
)

# Power usage per GPU

In [140]:
mean_grouping = carbondf.groupby(pd.Grouper(key = 'time', freq='10min')).mean()
mean_grouping['date']= mean_grouping.index
fig3= mean_grouping.plot(x="date", y=["GPU power"], kind="line")
fig3.update_layout(
    xaxis_title= "Date",
    yaxis_title="GPU Power Used (W)",
)
fig3.add_hline(y=mean_grouping['GPU power'].mean(), line_width=3, line_dash="dash", line_color="red",name='avg power')

/tmp/ipykernel_544372/3190066506.py:1: FutureWarning:

The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.



In [310]:
mean_grouping['GPU power'].describe()

count    2735.000000
mean     1664.541940
std       287.324016
min      1252.252438
25%      1468.927675
50%      1589.101406
75%      1757.977633
max      2690.133016
Name: GPU power, dtype: float64

### Final power consumption

In [311]:
alllogdf.iloc[-1]['date'] - alllogdf.iloc[0]['date']

Timedelta('17 days 03:40:00')

This is equal to 434 hours 

In [312]:
alllogdf.tail()

,date,timestamp,RAM energy,GPU energy,GPU power,CPU energy,Energy consumed,time,Energy by interval,requests,time
2022-10-09 18:20:00,2022-10-09 18:20:00,18:29:50,195.608696,649.189546,2524.044,17.479743,862.277986,2022-10-09 18:20:00,0.454074,133.0,2022-10-09 18:20:00
2022-10-09 18:30:00,2022-10-09 18:30:00,18:39:59,195.688778,649.607310,2532.620,17.486826,862.774974,2022-10-09 18:30:00,0.496988,174.0,2022-10-09 18:30:00
2022-10-09 18:40:00,2022-10-09 18:40:00,18:49:50,195.766173,650.009647,2488.633,17.493910,863.269730,2022-10-09 18:40:00,0.494756,160.0,2022-10-09 18:40:00
2022-10-09 18:50:00,2022-10-09 18:50:00,18:59:59,195.846260,650.424012,2521.772,17.500994,863.763391,2022-10-09 18:50:00,0.493661,213.0,2022-10-09 18:50:00
2022-10-09 19:00:00,2022-10-09 19:00:00,19:09:50,195.923680,650.817364,2454.498,17.508077,864.249121,2022-10-09 19:00:00,0.485730,125.0,2022-10-09 19:00:00


In [155]:
mean_grouping["GPU power"].describe()

count    2735.000000
mean     1664.541940
std       287.324016
min      1252.252438
25%      1468.927675
50%      1589.101406
75%      1757.977633
max      2690.133016
Name: GPU power, dtype: float64

In [141]:
min_grouping = carbondf.groupby(pd.Grouper(key = 'time', freq='10min')).min()
min_grouping['date']= min_grouping.index
fig4= min_grouping.plot(x="date", y=["GPU power"], kind="line")
fig4.update_layout(
    xaxis_title= "Date",
    yaxis_title="GPU Power Used (W)",
)

In [142]:
fig5= alllogdf.plot(kind="scatter", x="requests", y="Energy by interval")
fig5.update_layout(
    autosize=False,
    width=1000,
    height=500,
    xaxis_title= "Number of requests received in a 30-minute interval",
    yaxis_title="Energy consumption (kWh)",
)

### Trying to fit quadratic curve

In [364]:
x= alllogdf["requests"]
y= alllogdf["Energy by interval"]

In [ ]:
import numpy as np
from scipy.optimize import curve_fit
import matplotlib.pyplot as plt 

a, b, c = np.polyfit(x, y, 2)

fit_equation = lambda x: a * x ** 2 + b * x + c

def plot_fit(X, Y, f):
    X_fit = np.linspace(min(X), max(X), 1000)
    Y_fit = f(X_fit)

    fig, ax1 = plt.subplots()
    ax1.plot(X_fit, Y_fit, color='r', alpha=0.5, label='Polynomial fit')
    ax1.scatter(X, Y, s=4, color='b', label='Data points')
    ax1.set_title('Polynomial fit to data')
    ax1.legend()
    plt.show()
    
plot_fit(x, y, fit_equation)


#### Calculating intercept:

In [45]:
fit_equation(0)

0.28842679888794365